In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [3]:
params = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        #,'num_round': 1000
}

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [5]:
params_base = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'eta': 0.1
        ,'gamma': 0.0
        ,'alpha': 0.0
        ,'lambda': 1.0
        ,'min_child_weight': 6
        ,'max_depth': 1
        ,'subsample': 0.8
        ,'colsample_bytree': 0.8
        ,'random_state': 17
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        ,'eval_metric': 'auc'
        ,'colsample_bylevel': 0.3
        #,'num_round': 100000
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10, log=True)
            ,"max_depth": trial.suggest_int("max_depth", 1, 15, step=1, log=False)
            ,"subsample": trial.suggest_float("subsample", 0.1, 0.95,  step =0.05, log=False)
            ,"colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.95,  step =0.05, log=False)
            ,'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 0.95,  step =0.05, log=False)
            ,"gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            # 余裕があれば調整する
            ,"alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True)
            ,"lambda": trial.suggest_float("lambda", 1e-6, 10.0, log=True)
            
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = xgb.XGBClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## 特徴量生成

In [6]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 238)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [7]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [8]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [10]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
### 
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1172 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1126 features are remained after removing non-informative features
129 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 997 features are remained after removing features not interesting for LightGBM classifier
---=> 54 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1051 features

---=> df final shape: (356250, 1051)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [13]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


## 学習

### 001 baseline

In [14]:
# split train, and test datasets
df = df.replace([np.inf, -np.inf], np.nan)
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [18]:
app_train = app_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,1,1,0,0,0,0,0,0.051055
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,1,0,1,0,0,0,0,0.166067
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,0,1,0,1,1,0,1,0,0.043899
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,1,0,1,1,0,1,0,0.056542
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.183168


In [19]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [20]:
app_train = app_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [21]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [22]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [23]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,1,0,0,0,0,0,0.516856,0.162368
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,0,0,0,0.010645,0.027249
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,0,0,0,0.054077,0.030869
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,1,1,0,0,0,0.059492,0.029480
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,0,0,0,0.076165,0.047569


In [24]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [25]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1051) (61502, 1051)
[22:40:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79702	validation_1-auc:0.78408
[1]	validation_0-auc:0.80239	validation_1-auc:0.78716
[2]	validation_0-auc:0.80487	validation_1-auc:0.78758
[3]	validation_0-auc:0.80629	validation_1-auc:0.78783
[4]	validation_0-auc:0.80833	validation_1-auc:0.78763
[5]	validation_0-auc:0.80995	validation_1-auc:0.78684
[6]	validation_0-auc:0.81117	validation_1-auc:0.78660
[7]	validation_0-auc:0.81239	validation_1-auc:0.78659
[8]	validation_0-auc:0.81428	validation_1-auc:0.78743
[9]	val

[31]	validation_0-auc:0.84260	validation_1-auc:0.79123
[32]	validation_0-auc:0.84437	validation_1-auc:0.79109
[33]	validation_0-auc:0.84541	validation_1-auc:0.79094
[34]	validation_0-auc:0.84656	validation_1-auc:0.79106
[35]	validation_0-auc:0.84715	validation_1-auc:0.79101
[36]	validation_0-auc:0.84870	validation_1-auc:0.79114
[37]	validation_0-auc:0.84988	validation_1-auc:0.79124
[38]	validation_0-auc:0.85088	validation_1-auc:0.79140
[39]	validation_0-auc:0.85280	validation_1-auc:0.79149
[40]	validation_0-auc:0.85421	validation_1-auc:0.79162
[41]	validation_0-auc:0.85612	validation_1-auc:0.79172
[42]	validation_0-auc:0.85768	validation_1-auc:0.79186
[43]	validation_0-auc:0.85936	validation_1-auc:0.79186
[44]	validation_0-auc:0.86087	validation_1-auc:0.79192
[45]	validation_0-auc:0.86207	validation_1-auc:0.79181
[46]	validation_0-auc:0.86333	validation_1-auc:0.79181
[47]	validation_0-auc:0.86453	validation_1-auc:0.79186
[48]	validation_0-auc:0.86591	validation_1-auc:0.79198
[49]	valid

[71]	validation_0-auc:0.88404	validation_1-auc:0.78576
[72]	validation_0-auc:0.88491	validation_1-auc:0.78589
[73]	validation_0-auc:0.88596	validation_1-auc:0.78595
[74]	validation_0-auc:0.88734	validation_1-auc:0.78597
[75]	validation_0-auc:0.88833	validation_1-auc:0.78599
[76]	validation_0-auc:0.88867	validation_1-auc:0.78593
[77]	validation_0-auc:0.88966	validation_1-auc:0.78596
[78]	validation_0-auc:0.89056	validation_1-auc:0.78596
[79]	validation_0-auc:0.89090	validation_1-auc:0.78605
[80]	validation_0-auc:0.89147	validation_1-auc:0.78604
[81]	validation_0-auc:0.89237	validation_1-auc:0.78615
[82]	validation_0-auc:0.89325	validation_1-auc:0.78612
[83]	validation_0-auc:0.89368	validation_1-auc:0.78612
[84]	validation_0-auc:0.89401	validation_1-auc:0.78613
[85]	validation_0-auc:0.89462	validation_1-auc:0.78603
[86]	validation_0-auc:0.89525	validation_1-auc:0.78598
[87]	validation_0-auc:0.89613	validation_1-auc:0.78604
[88]	validation_0-auc:0.89642	validation_1-auc:0.78596
[89]	valid

[1]	validation_0-auc:0.80033	validation_1-auc:0.79178
[2]	validation_0-auc:0.80300	validation_1-auc:0.79299
[3]	validation_0-auc:0.80475	validation_1-auc:0.79357
[4]	validation_0-auc:0.80654	validation_1-auc:0.79340
[5]	validation_0-auc:0.80842	validation_1-auc:0.79266
[6]	validation_0-auc:0.80916	validation_1-auc:0.79258
[7]	validation_0-auc:0.81040	validation_1-auc:0.79233
[8]	validation_0-auc:0.81220	validation_1-auc:0.79277
[9]	validation_0-auc:0.81368	validation_1-auc:0.79297
[10]	validation_0-auc:0.81518	validation_1-auc:0.79283
[11]	validation_0-auc:0.81656	validation_1-auc:0.79302
[12]	validation_0-auc:0.81767	validation_1-auc:0.79307
[13]	validation_0-auc:0.81873	validation_1-auc:0.79293
[14]	validation_0-auc:0.82010	validation_1-auc:0.79320
[15]	validation_0-auc:0.82184	validation_1-auc:0.79329
[16]	validation_0-auc:0.82348	validation_1-auc:0.79315
[17]	validation_0-auc:0.82519	validation_1-auc:0.79299
[18]	validation_0-auc:0.82667	validation_1-auc:0.79294
[19]	validation_0-a

In [26]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
485,EXT_SOURCES_NANMEDIAN,0.016188,0.000819
483,EXT_SOURCES_MEAN,0.012166,0.000414
1050,pred_1b,0.011011,0.000515
1049,pred_1a,0.009572,0.000142
134,APPS_EXT_SOURCE_MEAN,0.007781,0.000911
491,EXT_SOURCE_3,0.004608,0.001984
490,EXT_SOURCE_2,0.004593,0.001347
145,APP_EXT_SOURCE_2_EXT_SOURCE_3,0.003982,0.000207
484,EXT_SOURCES_MIN,0.003553,0.000347
482,EXT_SOURCES_MAX,0.002549,0.000153


In [27]:
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [28]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,1,0,0,0,0,0,0.516856,0.162368
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,0,0,0,0.010645,0.027249
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,0,0,0,0.054077,0.030869
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,1,1,0,0,0,0.059492,0.029480
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,0,0,0,0.076165,0.047569


In [29]:
# データセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [30]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [31]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032806
1,100005,0.131737
2,100013,0.036010
3,100028,0.031898
4,100038,0.196477


### 正規化＋対数変換

In [32]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [33]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [34]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,1,0,0,0,0,0,0.516856,0.162368
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,0,0,0,0.010645,0.027249
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,0,0,0,0.054077,0.030869
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,1,1,0,0,0,0.059492,0.029480
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,0,0,0,0.076165,0.047569


In [35]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.038227
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166067,0.112768
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043899,0.033638
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056542,0.031213
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.181109


In [36]:
# infをnanへ置換
app_train_ord = FeatureEngineering2.reduce_mem_usage(app_train_ord)
app_test_ord = FeatureEngineering2.reduce_mem_usage(app_test_ord)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051056,0.038239
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166016,0.112793
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043884,0.033630
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056549,0.031219
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183228,0.181152


In [37]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [38]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])

app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051056,0.038239
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,0,0,0,0.166016,0.112793
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,1,0,1,0,0.043884,0.033630
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,1,0,1,0,0.056549,0.031219
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183228,0.181152


In [39]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,1,0,0,0,0,0,0.051056,0.038239
100005,1,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,0,0,0,0.166016,0.112793
100013,2,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,0,1,1,0,1,0,0.043884,0.033630
100028,3,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,1,0,1,0,0.056549,0.031219
100038,4,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,1,0,0,0,0,0,0.183228,0.181152


In [40]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.369385,-0.423584
100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0,0.969238,0.423340
100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,-0.452881,-0.476074
100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,-0.305420,-0.503418
100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,1.169922,1.200195


In [41]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,pred_1a
0,100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,-0.369385,-0.423584
1,100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0,0.969238,0.423340
2,100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,-0.452881,-0.476074
3,100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0,-0.305420,-0.503418
4,100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0,1.169922,1.200195


In [42]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [43]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [44]:
#del app_train_ord
#del app_test_ord
#gc.collect()

0

In [45]:
### 学習モデル構築
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1051) (61502, 1051)
[22:42:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[3]	validation_0-auc:0.80643	validation_1-auc:0.78811
[4]	validation_0-auc:0.80868	validation_1-auc:0.78776
[5]	validation_0-auc:0.81033	validation_1-auc:0.78722
[6]	validation_0-auc:0.81160	validation_1-auc:0.78690
[7]	validation_0-auc:0.81266	validation_1-auc:0.78665
[8]	validation_0-auc:0.81426	validation_1-auc:0.78754
[9]	val

[31]	validation_0-auc:0.84168	validation_1-auc:0.79150
[32]	validation_0-auc:0.84327	validation_1-auc:0.79140
[33]	validation_0-auc:0.84448	validation_1-auc:0.79124
[34]	validation_0-auc:0.84577	validation_1-auc:0.79119
[35]	validation_0-auc:0.84629	validation_1-auc:0.79113
[36]	validation_0-auc:0.84773	validation_1-auc:0.79153
[37]	validation_0-auc:0.84888	validation_1-auc:0.79179
[38]	validation_0-auc:0.84987	validation_1-auc:0.79182
[39]	validation_0-auc:0.85188	validation_1-auc:0.79197
[40]	validation_0-auc:0.85361	validation_1-auc:0.79180
[41]	validation_0-auc:0.85450	validation_1-auc:0.79179
[42]	validation_0-auc:0.85605	validation_1-auc:0.79169
[43]	validation_0-auc:0.85740	validation_1-auc:0.79170
[44]	validation_0-auc:0.85876	validation_1-auc:0.79182
[45]	validation_0-auc:0.85994	validation_1-auc:0.79180
[46]	validation_0-auc:0.86126	validation_1-auc:0.79173
[47]	validation_0-auc:0.86255	validation_1-auc:0.79177
[48]	validation_0-auc:0.86365	validation_1-auc:0.79181
[49]	valid

[71]	validation_0-auc:0.88399	validation_1-auc:0.78531
[72]	validation_0-auc:0.88483	validation_1-auc:0.78524
[73]	validation_0-auc:0.88555	validation_1-auc:0.78520
[74]	validation_0-auc:0.88659	validation_1-auc:0.78524
[75]	validation_0-auc:0.88754	validation_1-auc:0.78524
[76]	validation_0-auc:0.88790	validation_1-auc:0.78522
[77]	validation_0-auc:0.88850	validation_1-auc:0.78519
[78]	validation_0-auc:0.88973	validation_1-auc:0.78491
[79]	validation_0-auc:0.89008	validation_1-auc:0.78484
[80]	validation_0-auc:0.89055	validation_1-auc:0.78489
[81]	validation_0-auc:0.89108	validation_1-auc:0.78497
[82]	validation_0-auc:0.89191	validation_1-auc:0.78492
[83]	validation_0-auc:0.89242	validation_1-auc:0.78478
[84]	validation_0-auc:0.89323	validation_1-auc:0.78465
[85]	validation_0-auc:0.89373	validation_1-auc:0.78463
[86]	validation_0-auc:0.89432	validation_1-auc:0.78460
[87]	validation_0-auc:0.89488	validation_1-auc:0.78464
[88]	validation_0-auc:0.89552	validation_1-auc:0.78475
[89]	valid

[1]	validation_0-auc:0.80027	validation_1-auc:0.79170
[2]	validation_0-auc:0.80255	validation_1-auc:0.79238
[3]	validation_0-auc:0.80424	validation_1-auc:0.79305
[4]	validation_0-auc:0.80616	validation_1-auc:0.79303
[5]	validation_0-auc:0.80816	validation_1-auc:0.79223
[6]	validation_0-auc:0.80887	validation_1-auc:0.79242
[7]	validation_0-auc:0.81034	validation_1-auc:0.79232
[8]	validation_0-auc:0.81209	validation_1-auc:0.79271
[9]	validation_0-auc:0.81322	validation_1-auc:0.79289
[10]	validation_0-auc:0.81453	validation_1-auc:0.79252
[11]	validation_0-auc:0.81599	validation_1-auc:0.79305
[12]	validation_0-auc:0.81693	validation_1-auc:0.79299
[13]	validation_0-auc:0.81799	validation_1-auc:0.79273
[14]	validation_0-auc:0.81895	validation_1-auc:0.79320
[15]	validation_0-auc:0.82078	validation_1-auc:0.79331
[16]	validation_0-auc:0.82225	validation_1-auc:0.79314
[17]	validation_0-auc:0.82396	validation_1-auc:0.79310
[18]	validation_0-auc:0.82549	validation_1-auc:0.79293
[19]	validation_0-a

In [46]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
485,EXT_SOURCES_NANMEDIAN,0.015964,0.002255
483,EXT_SOURCES_MEAN,0.012186,0.000853
1050,pred_1b,0.010881,0.000479
1049,pred_1a,0.009541,0.000106
134,APPS_EXT_SOURCE_MEAN,0.007690,0.000663
490,EXT_SOURCE_2,0.005205,0.002293
491,EXT_SOURCE_3,0.004501,0.001976
145,APP_EXT_SOURCE_2_EXT_SOURCE_3,0.003907,0.000349
484,EXT_SOURCES_MIN,0.003643,0.000432
482,EXT_SOURCES_MAX,0.002679,0.000161


In [47]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [48]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSourceXgb2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.048181
1,100005,0.131152
2,100013,0.040155
3,100028,0.033271
4,100038,0.190030


## パラメータチューニング

In [49]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-07-06 22:44:40,156] A new study created in memory with name: no-name-4da78ab9-6c39-4280-b5d5-6696a30879ce


[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[0]	validation_0-auc:0.79654	validation_1-auc:0.78480
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[1]	validation_0-auc:0.80212	validation_1-auc:0.78756
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[2]	validation_0-auc:0.80504	validation_1-auc:0.78823
[3]	validation_0-auc:0.80643	validation_1-auc:0.78811
[3]	validation_0-auc:0.80643	validation_1-auc:0.78811
[3]	validation_0-auc:0.80643	validation_1-auc:0.78811
[3]	validation_0-auc:0.80643

[29]	validation_0-auc:0.84024	validation_1-auc:0.78989
[30]	validation_0-auc:0.84178	validation_1-auc:0.78951
[30]	validation_0-auc:0.84178	validation_1-auc:0.78951
[30]	validation_0-auc:0.84178	validation_1-auc:0.78951
[31]	validation_0-auc:0.84277	validation_1-auc:0.78924
[30]	validation_0-auc:0.84178	validation_1-auc:0.78951
[31]	validation_0-auc:0.84277	validation_1-auc:0.78924
[31]	validation_0-auc:0.84277	validation_1-auc:0.78924
[31]	validation_0-auc:0.84277	validation_1-auc:0.78924
[32]	validation_0-auc:0.84401	validation_1-auc:0.78893
[31]	validation_0-auc:0.84277	validation_1-auc:0.78924
[32]	validation_0-auc:0.84401	validation_1-auc:0.78893
[32]	validation_0-auc:0.84401	validation_1-auc:0.78893
[32]	validation_0-auc:0.84401	validation_1-auc:0.78893
[32]	validation_0-auc:0.84401	validation_1-auc:0.78893
[33]	validation_0-auc:0.84534	validation_1-auc:0.78886
[33]	validation_0-auc:0.84534	validation_1-auc:0.78886
[33]	validation_0-auc:0.84534	validation_1-auc:0.78886
[33]	valid

[59]	validation_0-auc:0.87701	validation_1-auc:0.78919
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[59]	validation_0-auc:0.87701	validation_1-auc:0.78919
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[63]	validation_0-auc:0.88048	validation_1-auc:0.78889
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[63]	validation_0-auc:0.88048	validation_1-auc:0.78889
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[64]	validation_0-auc:0.88126	validation_1-auc:0.78883
[63]	valid

[89]	validation_0-auc:0.89718	validation_1-auc:0.78893
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[88]	validation_0-auc:0.89593	validation_1-auc:0.78896
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[89]	validation_0-auc:0.89718	validation_1-auc:0.78893
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[93]	validation_0-auc:0.89980	validation_1-auc:0.78873
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[94]	validation_0-auc:0.90047	validation_1-auc:0.78863
[93]	validation_0-auc:0.89980	validation_1-auc:0.78873
[91]	valid

...

[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[58]	validation_0-auc:0.87619	validation_1-auc:0.78914
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[59]	validation_0-auc:0.87701	validation_1-auc:0.78919
[61]	validation_0-auc:0.87904	validation_1-auc:0.78901
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[63]	validation_0-auc:0.88048	validation_1-auc:0.78889
[63]	validation_0-auc:0.88048	validation_1-auc:0.78889
[62]	validation_0-auc:0.87985	validation_1-auc:0.78894
[60]	validation_0-auc:0.87796	validation_1-auc:0.78913
[63]	validation_0-auc:0.88048	validation_1-auc:0.78889
[64]	validation_0-auc:0.88126	validation_1-auc:0.78883
[63]	valid


[89]	validation_0-auc:0.89718	validation_1-auc:0.78893
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[87]	validation_0-auc:0.89571	validation_1-auc:0.78891
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889[92]	validation_0-auc:0.89891	validation_1-auc:0.78894

[88]	validation_0-auc:0.89593	validation_1-auc:0.78896
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[91]	validation_0-auc:0.89859	validation_1-auc:0.78889
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[93]	validation_0-auc:0.89980	validation_1-auc:0.78873
[89]	validation_0-auc:0.89718	validation_1-auc:0.78893
[92]	validation_0-auc:0.89891	validation_1-auc:0.78894
[93]	validation_0-auc:0.89980	validation_1-auc:0.78873
[93]	validation_0-auc:0.89980	validation_1-auc:0.78873
[94]	validation_0-auc:0.90047	validation_1-auc:0.78863
[90]	validation_0-auc:0.89771	validation_1-auc:0.78889
[93]	vali

[I 2022-07-06 22:59:07,210] Trial 45 finished with value: 0.7888061256073766 and parameters: {'min_child_weight': 7.488693180363583, 'max_depth': 6, 'subsample': 0.95, 'colsample_bytree': 0.25, 'colsample_bylevel': 0.7000000000000001, 'gamma': 0.00019408633875590522, 'alpha': 8.304450410197197e-08, 'lambda': 5.359760212821288e-05}. Best is trial 0 with value: 0.7888061256073766.


[99]	validation_0-auc:0.90384	validation_1-auc:0.78881


[I 2022-07-06 22:59:08,330] Trial 46 finished with value: 0.7888061256073766 and parameters: {'min_child_weight': 8.366324882338832, 'max_depth': 6, 'subsample': 0.35, 'colsample_bytree': 0.25, 'colsample_bylevel': 0.75, 'gamma': 0.00018825386743802434, 'alpha': 1.6985610903058181e-07, 'lambda': 7.825077393666595e-05}. Best is trial 0 with value: 0.7888061256073766.
[I 2022-07-06 22:59:08,618] Trial 47 finished with value: 0.7888061256073766 and parameters: {'min_child_weight': 4.784343798695868, 'max_depth': 6, 'subsample': 0.95, 'colsample_bytree': 0.25, 'colsample_bylevel': 0.75, 'gamma': 0.0002905598021810697, 'alpha': 6.879821603394709e-06, 'lambda': 7.260599129107585e-05}. Best is trial 0 with value: 0.7888061256073766.
[I 2022-07-06 22:59:08,968] Trial 48 finished with value: 0.7888061256073766 and parameters: {'min_child_weight': 2.280367202989038, 'max_depth': 9, 'subsample': 0.35, 'colsample_bytree': 0.4, 'colsample_bylevel': 0.75, 'gamma': 0.0003644053944630628, 'alpha': 5.9

In [50]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7888


{'min_child_weight': 0.9583499638223313,
 'max_depth': 10,
 'subsample': 0.85,
 'colsample_bytree': 0.30000000000000004,
 'colsample_bylevel': 0.30000000000000004,
 'gamma': 3.969548650285048e-06,
 'alpha': 3.331495079988004e-06,
 'lambda': 5.371445879448848e-05}

In [51]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'min_child_weight': 6,
 'max_depth': 1,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'colsample_bylevel': 0.3,
 'gamma': 0.0,
 'alpha': 0.0,
 'lambda': 1.0,
 'booster': 'gbtree',
 'objective': 'binary:logistic',
 'eta': 0.1,
 'random_state': 17,
 'gpu_id': 0,
 'tree_method': 'gpu_hist',
 'eval_metric': 'auc'}

In [52]:
train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246004, 1051) (61502, 1051)
[0]	validation_0-auc:0.66139	validation_1-auc:0.66097
[1]	validation_0-auc:0.69704	validation_1-auc:0.69616
[2]	validation_0-auc:0.71437	validation_1-auc:0.71400
[3]	validation_0-auc:0.72129	validation_1-auc:0.72089
[4]	validation_0-auc:0.72392	validation_1-auc:0.72441
[5]	validation_0-auc:0.72253	validation_1-auc:0.72361
[6]	validation_0-auc:0.72931	validation_1-auc:0.72895
[7]	validation_0-auc:0.72933	validation_1-auc:0.72947
[8]	validation_0-auc:0.72756	validation_1-auc:0.72809
[9]	validation_0-auc:0.72623	validation_1-auc:0.72667
[10]	validation_0-auc:0.73244	validation_1-auc:0.73258
[11]	validation_0-auc:0.73249	validation_1-auc:0.73341
[12]	validation_0-auc:0.73100	validation_1-auc:0.73230
[13]	validation_0-auc:0.73037	validation_1-auc:0.73181
[14]	validation_0-auc:0.73226	validation_1-auc:0.73445
[15]	validation_0-auc:0.73176	validation_1-auc:0.73361
[16]	validation_0-auc:0.73100	validation_1-auc:0.73288
[1

[47]	validation_0-auc:0.77350	validation_1-auc:0.77325
[48]	validation_0-auc:0.77431	validation_1-auc:0.77398
[49]	validation_0-auc:0.77443	validation_1-auc:0.77418
[50]	validation_0-auc:0.77461	validation_1-auc:0.77437
[51]	validation_0-auc:0.77475	validation_1-auc:0.77447
[52]	validation_0-auc:0.77496	validation_1-auc:0.77465
[53]	validation_0-auc:0.77519	validation_1-auc:0.77488
[54]	validation_0-auc:0.77531	validation_1-auc:0.77490
[55]	validation_0-auc:0.77572	validation_1-auc:0.77539
[56]	validation_0-auc:0.77611	validation_1-auc:0.77577
[57]	validation_0-auc:0.77629	validation_1-auc:0.77591
[58]	validation_0-auc:0.77644	validation_1-auc:0.77599
[59]	validation_0-auc:0.77655	validation_1-auc:0.77603
[60]	validation_0-auc:0.77777	validation_1-auc:0.77728
[61]	validation_0-auc:0.77790	validation_1-auc:0.77735
[62]	validation_0-auc:0.77873	validation_1-auc:0.77819
[63]	validation_0-auc:0.77881	validation_1-auc:0.77822
[64]	validation_0-auc:0.77904	validation_1-auc:0.77844
[65]	valid

[95]	validation_0-auc:0.78555	validation_1-auc:0.77903
[96]	validation_0-auc:0.78558	validation_1-auc:0.77907
[97]	validation_0-auc:0.78581	validation_1-auc:0.77927
[98]	validation_0-auc:0.78587	validation_1-auc:0.77933
[99]	validation_0-auc:0.78608	validation_1-auc:0.77951
[auc] tr:0.786078, va0.7795
-------------------- 3 --------------------
(246005, 1051) (61501, 1051)
[0]	validation_0-auc:0.66199	validation_1-auc:0.65856
[1]	validation_0-auc:0.70312	validation_1-auc:0.70172
[2]	validation_0-auc:0.71672	validation_1-auc:0.71705
[3]	validation_0-auc:0.71877	validation_1-auc:0.71985
[4]	validation_0-auc:0.72416	validation_1-auc:0.72495
[5]	validation_0-auc:0.72298	validation_1-auc:0.72367
[6]	validation_0-auc:0.72424	validation_1-auc:0.72478
[7]	validation_0-auc:0.72676	validation_1-auc:0.72714
[8]	validation_0-auc:0.72695	validation_1-auc:0.72687
[9]	validation_0-auc:0.72552	validation_1-auc:0.72546
[10]	validation_0-auc:0.73029	validation_1-auc:0.73140
[11]	validation_0-auc:0.72908

[41]	validation_0-auc:0.76534	validation_1-auc:0.76646
[42]	validation_0-auc:0.76623	validation_1-auc:0.76746
[43]	validation_0-auc:0.76869	validation_1-auc:0.77012
[44]	validation_0-auc:0.77036	validation_1-auc:0.77196
[45]	validation_0-auc:0.77107	validation_1-auc:0.77269
[46]	validation_0-auc:0.77112	validation_1-auc:0.77290
[47]	validation_0-auc:0.77197	validation_1-auc:0.77386
[48]	validation_0-auc:0.77216	validation_1-auc:0.77406
[49]	validation_0-auc:0.77254	validation_1-auc:0.77449
[50]	validation_0-auc:0.77274	validation_1-auc:0.77465
[51]	validation_0-auc:0.77293	validation_1-auc:0.77477
[52]	validation_0-auc:0.77309	validation_1-auc:0.77490
[53]	validation_0-auc:0.77333	validation_1-auc:0.77499
[54]	validation_0-auc:0.77357	validation_1-auc:0.77520
[55]	validation_0-auc:0.77414	validation_1-auc:0.77589
[56]	validation_0-auc:0.77563	validation_1-auc:0.77710
[57]	validation_0-auc:0.77576	validation_1-auc:0.77719
[58]	validation_0-auc:0.77595	validation_1-auc:0.77731
[59]	valid

In [54]:
### 推論用データセットの作成
x_test = app_test.drop(columns=['SK_ID_CURR'])
id_test = app_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

### predict
test_pred = baseline.predict_xgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

### make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_XgbHyperParameterTuningXgb2-3.csv", index=None)

-------------------- 0 --------------------


ValueError: Feature shape mismatch, expected: 1051, got 1052

In [ ]:
del app_train_ord
del app_test_ord
gc.collect()